# Transfer Learning
---
## 쉽지만 중요한 코너!

**잘 만들어진** 모델들을 가져다가 **고쳐** 사용해보자.

* [모델들은 이 링크를 참고해보자](https://keras.io/applications/)
* [트랜스퍼 러닝?](https://miro.medium.com/max/2800/1*D5S6ylZwUZAxj0lMyipZ2g.png)

![transfer](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/05/31112715/finetune1.jpg)


## 라이브러리 로딩

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import decode_predictions
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

from sklearn.model_selection import train_test_split

import random
import numpy as np
import matplotlib.pyplot as plt
import glob

## 데이터 수집합시다!

* 최소 조건 : 클래스 3개, 한 클래스당 10장 이상. **다다익선!**
    * image-net data에는 확실히 없을만한 것들로.
    * **좋은 결과**를 위해서라면 확실히 차이나는 것들로.
    * **도전(역경)**을 위해서라면 클래스가 달라도 비슷비슷 한걸로.
---
* **본인의** 구글 드라이브 my_data 폴더에 클래스당 각각 폴더를 만들어서 업로드.
- **상세 설명**
    1. my_data 폴더를 **본인의** 구글 드라이브 바로 아래에 만들어둔다.
    2. my_data 폴더 안에 transfer 폴더를 만든다.
    3. transfer 폴더 안에 이미지들을 수집하고, 하나의 클래스당 하나의 폴더를 갖도록 정리/업로드한다.
    4. 5초 정도 여유를 갖자.
    5. 아래 코드들을 실행한다

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cd /content/drive/MyDrive/my_data/; ls

In [ ]:
files = glob.glob('/content/drive/MyDrive/my_data/transfer/*/*')
files

In [ ]:
name_cnt = {}

for x in files :
    name_cnt[x.split('/')[-2]] = name_cnt.get(x.split('/')[-2], 0) + 1

name_cnt

In [ ]:
i = 0
names = {}

for key in name_cnt :
    names[key] = i     # names_cnt의 key값에 새로운 값 부여
    i += 1             # 클래스 수만큼 i값 증가

names

In [ ]:
images = []
labels = []

for path in files:
    img = image.load_img(path, target_size=(299,299) )
    img = image.img_to_array(img)
    
    images.append(img)
    labels.append(names[path.split('/')[-2]])
    
    plt.imshow(image.load_img(path))
    plt.show()

images_arr = np.array(images)
labels_arr = np.array(labels)

print(images_arr.shape)
print(labels_arr.shape)

In [ ]:
print(labels_arr)
label_v = len(np.unique(labels_arr))
label_v

In [ ]:
### 라벨링
y = to_categorical(labels, label_v)

In [ ]:
print(y[:3])
y.shape

## 데이터를 나누자, train-valid-test

- 각 이미지 그룹별로 균등한 분할을 위하여. 아래 코드가 조금 복잡하다.

In [ ]:
temp = []
init_v = 0

for v in name_cnt.values() :
    temp.append( (images[init_v:init_v+v], y[init_v:init_v+v]) )
    init_v += v

In [ ]:
for i in range(len(temp)) :
    x_to_array = np.array(temp[i][0])
    y_to_array = np.array(temp[i][1])

    train_x, test_x, train_y, test_y =\
        train_test_split(x_to_array, y_to_array, test_size=0.2, random_state=2023)
    
    train_x, valid_x, train_y, valid_y =\
        train_test_split(train_x, train_y, test_size=0.2, random_state=2023)

    if i==0 :
        first_tr_x, first_va_x, first_te_x = train_x.copy(), valid_x.copy(), test_x.copy()
        first_tr_y, first_va_y, first_te_y = train_y.copy(), valid_y.copy(), test_y.copy()

    elif i==1 :
        new_tr_x, new_tr_y = np.vstack((first_tr_x, train_x)), np.vstack((first_tr_y, train_y))
        new_va_x, new_va_y = np.vstack((first_va_x, valid_x)), np.vstack((first_va_y, valid_y))
        new_te_x, new_te_y = np.vstack((first_te_x, test_x)), np.vstack((first_te_y, test_y))

    else :
        new_tr_x, new_tr_y = np.vstack((new_tr_x, train_x)), np.vstack((new_tr_y, train_y))
        new_va_x, new_va_y = np.vstack((new_va_x, valid_x)), np.vstack((new_va_y, valid_y))
        new_te_x, new_te_y = np.vstack((new_te_x, test_x)), np.vstack((new_te_y, test_y))

In [ ]:
new_tr_x.shape, new_tr_y.shape, new_va_x.shape, new_va_y.shape, new_te_x.shape, new_te_y.shape

In [ ]:
# 전처리 하지 않은 파일 따로 시각화 해두기
train_xv, valid_xv, test_xv = train_x.copy(), valid_x.copy(), test_x.copy()

In [ ]:
new_tr_x.max(), new_tr_x.min()

In [ ]:
new_tr_x = preprocess_input(new_tr_x)
new_va_x = preprocess_input(new_va_x)
new_te_x = preprocess_input(new_te_x)

In [ ]:
new_tr_x.max(), new_tr_x.min()

## 인셉션! 남의 모델 불러오자!

In [ ]:
keras.backend.clear_session()

base_model = InceptionV3(weights='imagenet',       # 이미지넷 데이터 바탕으로 미리 학습된 것
                         include_top=False,        # 마지막 레이어 빼고!
                         input_shape= (299,299,3)) # 입력 데이터의 형태

new_output = GlobalAveragePooling2D()(base_model.output)
new_output = Dense(3, # class 3개   클래스 개수만큼 진행한다.
                  activation = 'softmax')(new_output)

model = keras.models.Model(base_model.inputs, new_output)

model.summary()

In [ ]:
print(f'모델의 레이어 수 : {len(model.layers)}')

## 이어서 학습시킬 레이어와, 고정시킬 레이어를 결정하자

In [ ]:
len(model.layers)

In [ ]:
for idx, layer in enumerate(model.layers) :
    if idx < 213 :
        layer.trainable = True
    else :
        layer.trainable = False

In [ ]:
# 처음부터 학습시키는 것도 아니고,
# 마지막 100개만 튜닝 할 것이므로 learning rate를 조금 크게 잡아본다.

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(lr=0.001) )

## Data Augmentation & Callbacks

In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                 patience=4,
                                 verbose=1,
                                 factor=0.5,
                                 min_lr=0.000001)

es = EarlyStopping(monitor='val_loss',
                   min_delta=0, # 개선되고 있다고 판단하기 위한 최소 변화량
                   patience=15, # 개선 없는 epoch 얼마나 기달려 줄거야
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=180, # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.3, # Randomly zoom image 
    width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(train_x)

## 학습 시켜본다!!

In [ ]:
# 데이터를 넣어서 학습시키자!
hist = model.fit(datagen.flow(train_x, train_y),
                 epochs=1000, validation_data=(valid_x, valid_y),
                 verbose=1, callbacks=[es, lr_reduction] )

## 결과를 본다!!

In [ ]:
model.evaluate(test_x, test_y) ## [loss, accuracy]

In [ ]:
y_pred = model.predict(test_x)
y_pred

In [ ]:
to_names = { v:k for k,v in names.items() }

In [ ]:
for i in range(len(test_x)) :
    print('------------------------------------------------------')
    print(f'실제 정답 : {to_names[test_y[i].argmax()]} vs 모델의 예측 : {to_names[y_pred[i].argmax()]} ')
    prob = ''
    
    for j in to_names :
        string = f'{to_names[j]} : {y_pred[i][j]*100:.2f}%  '
        prob = prob + string
    print(prob)
    plt.imshow(test_xv[i].reshape([299,299,3])/255)
    plt.show()